#DATASCI W261, Machine Learning at Scale
--------
####Assignement:  week \#2
####Lei Yang (leiyang@berkeley.edu)
####Due: 2016-01-26, 8AM PST

###*HW2.0.*
- What is a race condition in the context of parallel computation? Give an example.
- What is MapReduce?
- How does it differ from Hadoop?
- Which programming paradigm is Hadoop based on? Explain and give a simple example in code and show the code running.


####<span style="color:red">HW2.0 Answer:</span>


###start yarn and hdfs

In [2]:
!/usr/local/Cellar/hadoop/2*/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2*/sbin/start-dfs.sh

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-leiyang-resourcemanager-Leis-MacBook-Pro.local.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-leiyang-nodemanager-Leis-MacBook-Pro.local.out
16/01/23 10:11:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-leiyang-namenode-Leis-MacBook-Pro.local.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-leiyang-datanode-Leis-MacBook-Pro.local.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-leiyang-secondarynamenode-Leis-MacBook-Pro.local.out
16/01/23 10:11:28 WARN util.NativeCodeLoader: Unable to 

###create a folder and upload enronemail_1h.txt to hdfs

In [25]:
!hdfs dfs -mkdir -p /user/lei
!hdfs dfs -put enronemail_1h.txt /user/lei

16/01/23 10:56:17 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 10:56:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


###*HW2.1.* Sort in Hadoop MapReduce
- Given as input: Records of the form <integer, “NA”>, where integer is any integer, and “NA” is just the empty string.
- Output: sorted key value pairs of the form <integer, “NA”> in decreasing order; what happens if you have multiple reducers? Do you need additional steps? Explain.

- Write code to generate N  random records of the form <integer, “NA”>. Let N = 10,000.
- Write the python Hadoop streaming map-reduce job to perform this sort. Display the top 10 biggest numbers. Display the 10 smallest numbers

####<span style="color:red">HW2.1 Answer:</span>



###*HW2.2.*  WORDCOUNT
- Using the Enron data from HW1 and Hadoop MapReduce streaming, write the mapper/reducer job that  will determine the word count (number of occurrences) of each white-space delimitted token (assume spaces, fullstops, comma as delimiters). 
- Examine the word “assistance” and report its word count results.
- CROSSCHECK: >grep assistance enronemail_1h.txt|cut -d$'\t' -f4| grep assistance|wc -l    
  - 8    
  - \#NOTE  "assistance" occurs on 8 lines but how many times does the token occur? 10 times! This is the number we are looking for!

###Mapper

In [106]:
%%writefile mapper.py
#!/usr/bin/python
import sys, re, string
# define regex for punctuation removal
regex = re.compile('[%s]' % re.escape(string.punctuation))
# input comes from STDIN (standard input)
for line in sys.stdin:
    # use subject and body
    line = line.split('\t', 2)[-1]
    # remove punctuations, only have white-space as delimiter
    line = regex.sub(' ', line.lower())
    # split the line into words
    words = line.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        print '%s\t%s' % (word, 1)

Overwriting mapper.py


###Reducer

In [107]:
%%writefile reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None
wordcount = {}

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # save count            
            wordcount[current_word] = current_count
        current_count = count
        current_word = word

# do not forget to save the last word count if needed!
if current_word == word:    
    wordcount[current_word] = current_count
    
# found count for word assistance
findword = 'assistance'
print '%s\t%d' %(findword, wordcount[findword] if findword in wordcount else 0)

Overwriting reducer.py


###Run the job with Hadoop streaming

In [108]:
!hdfs dfs -rm -r results
!hadoop jar /usr/local/Cellar/hadoop/2.*/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar -mapper mapper.py -reducer reducer.py -input /user/lei/enronemail_1h.txt -output results

16/01/23 13:21:37 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 13:21:37 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted results
16/01/23 13:21:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 13:21:39 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/23 13:21:39 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/23 13:21:39 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/23 13:21:40 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/23 13:21:40 INFO mapreduce.JobSubmitter: number of splits:1
16/01/23 13:21:40 INFO mapreduce.JobSubmitter: Submitting 

###<span style="color:red">HW2.2 Results:</span>

In [109]:
!hdfs dfs -cat results/part-00000

16/01/23 13:21:45 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
assistance	10


###*HW2.2.1*  Using Hadoop MapReduce and your wordcount job (from HW2.2) determine the top-10 occurring tokens (most frequent tokens)

###Mapper - unchanged
###Reducer - same as above, only sort total count at the end

In [110]:
%%writefile reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys, operator

current_word = None
current_count = 0
word = None
wordcount = {}

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # save count            
            wordcount[current_word] = current_count
        current_count = count
        current_word = word

# do not forget to save the last word count if needed!
if current_word == word:    
    wordcount[current_word] = current_count
    
# sort count top get top n counts:
n = 10
sort_count = sorted(wordcount.items(), key=operator.itemgetter(1))
print 'Top %d counts out of %d words:' %(n, len(sort_count))
for i in range(n):
    print '%s\t%d' %(sort_count[-i-1])

Overwriting reducer.py


###Run the job with Hadoop streaming

In [111]:
!hdfs dfs -rm -r results
!hadoop jar /usr/local/Cellar/hadoop/2.*/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar -mapper mapper.py -reducer reducer.py -input /user/lei/enronemail_1h.txt -output results

16/01/23 13:22:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 13:22:07 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted results
16/01/23 13:22:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 13:22:09 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/23 13:22:09 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/23 13:22:09 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/23 13:22:09 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/23 13:22:09 INFO mapreduce.JobSubmitter: number of splits:1
16/01/23 13:22:09 INFO mapreduce.JobSubmitter: Submitting 

###<span style="color:red">HW2.2.1 Results:</span>

In [112]:
!hdfs dfs -cat results/part-00000

16/01/23 13:22:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Top 10 counts out of 5408 words:	
the	1247
to	964
and	670
of	566
a	543
you	445
in	418
your	395
ect	382
for	374


###*HW2.3.* Multinomial NAIVE BAYES with NO Smoothing
- Using the Enron data from HW1 and Hadoop MapReduce, write  a mapper/reducer job(s) that will both learn  Naive Bayes classifier and classify the Enron email messages using the learnt Naive Bayes classifier. 
- Use all white-space delimitted tokens as independent input variables (assume spaces, fullstops, commas as delimiters). 
- Note: for multinomial Naive Bayes, the Pr(X=“assistance”|Y=SPAM) is calculated as follows:
 - the number of times “assistance” occurs in SPAM labeled documents / the number of words in documents labeled SPAM 
 - E.g.,   “assistance” occurs 5 times in all of the documents Labeled SPAM, and the length in terms of the number of words in all documents labeled as SPAM (when concatenated) is 1,000. Then Pr(X=“assistance”|Y=SPAM) = 5/1000. 
 - Note this is a multinomial estimation of the class conditional for a Naive Bayes Classifier. No smoothing is needed in this HW. 
 - Multiplying lots of probabilities, which are between 0 and 1, can result in floating-point underflow. Since log(xy) = log(x) + log(y), it is better to perform all computations by summing logs of probabilities rather than multiplying probabilities. 
 - Please pay attention to probabilites that are zero! They will need special attention. Count up how many times you need to process a zero probabilty for each class and report. 
- Report the performance of your learnt classifier in terms of misclassifcation error rate of your multinomial Naive Bayes Classifier. Plot a histogram of the log posterior probabilities (i.e., log(Pr(Class|Doc))) for each class over the training set. Summarize what you see. 
- Error Rate = misclassification rate with respect to a provided set (say training set in this case). It is more formally defined here:
 - Let DF represent the evalution set in the following:
 - Err(Model, DF) = |{(X, c(X)) ∈ DF : c(X) != Model(x)}|   / |DF|
 - Where || denotes set cardinality; c(X) denotes the class of the tuple X in DF; and Model(X) denotes the class inferred by the Model “Model”

   

###Mapper (training)
- Differ from previous one, in that we need to separate counts for spam and ham
- key~value pair: word~[1, flag for ham/spam]

In [175]:
%%writefile mapper_t.py
#!/usr/bin/python
import sys, re, string
# define regex for punctuation removal
regex = re.compile('[%s]' % re.escape(string.punctuation))
# input comes from STDIN (standard input)
for line in sys.stdin:
    # use subject and body
    msg = line.split('\t', 2)
    if len(msg) < 3:
        continue
    isSpam = msg[1]
    # remove punctuations, only have white-space as delimiter
    words = regex.sub(' ', msg[-1].lower())
    # split the line into words
    words = words.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        print '%s\t%d\t%s' % (word, 1, isSpam)

Overwriting mapper_t.py


###Reducer (training)
- collapse word counts for spam and ham separately
- estimate NB model parameters: prior and conditional probabilities
- classify messages that contains the keyword
- **Note:** for messages that don't contain the keyword, the decision is solely based on prior probability, which will always give non-spam prediction, thus we skip those messages and only focus on those with the specified keyword
- output results

####Parameter estimation background:
Assuming *positional independence*, and without smoothing, the multinomial NB conditional probability $P(t | c)$ can be estimated as:
$$
\hat{P}(t\mid c)=\frac{T_{ct}+0}{(\sum_{t^\prime \in V}{T_{ct^\prime}})+B},
$$

where $B=|V|$ is the number of terms in the vocabulary $V$ (including all text classes), and $T_{ct}$ is the count of word *t* in class *c*. 

To classify a message, the posterior probability of class $c$ can be calculated as:
$$
c_{map}=\arg\max_{c\in\mathbb C}[\log{\hat{P}(c)}+\sum_{1\leqslant k \leqslant n_d}{\log{\hat{P}(t_k\mid c)}}],
$$
where $\hat{P}(t_k\mid c)$ is estimated above with *positional independence* assumption as $\hat{P}(t\mid c)$.

In [176]:
%%writefile reducer_t.py
#!/usr/bin/python
from operator import itemgetter
import sys, operator
import numpy as np

current_word = None
smooth_factor = 0 # no smoothing
current_count = [smooth_factor, smooth_factor]
word = None
wordcount = {}

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count, isSpam = line.split('\t', 2)

    # convert count and spam flag (currently a string) to int
    try:
        count = int(count)
        isSpam = int(isSpam)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:        
        current_count[isSpam] += count
    else:
        if current_word:
            # count finish for one word, save it
            wordcount[current_word] = current_count
        # initialize new count for new word
        current_count = [smooth_factor, smooth_factor]
        current_count[isSpam] = count
        current_word = word

# do not forget to save the last word count if needed!
if current_word == word:    
    wordcount[current_word] = current_count
    
# calculate NB parameters, and write the dictionary to a file for the classification job
n_total = np.sum(wordcount.values(), 0)
#print 'total count %s' %(str(n_total))

#print probability
for (key,value) in zip(wordcount.keys(), wordcount.values()/(1.0*n_total)):
    print '%s\t%s\t%s' %(key, value[0], value[1])

Overwriting reducer_t.py


###Mapper (classification)
- key~value pair: msgID~[probability, flag]

In [188]:
%%writefile mapper_c.py
#!/usr/bin/python
import sys, re, string, subprocess
# read the probability from HDFS
prob = {}
cat = subprocess.Popen(["hadoop", "fs", "-cat", "prob/part-00000"], stdout=subprocess.PIPE)
for line in cat.stdout:
    word, p0, p1 = line.split()
    prob[word] = [p0, p1]

# define regex for punctuation removal
regex = re.compile('[%s]' % re.escape(string.punctuation))
# input comes from STDIN (standard input)
for line in sys.stdin:
    # use subject and body
    msg = line.split('\t', 2)
    # skip bad message 
    if len(msg) < 3:
        continue
    msgID, isSpam = msg[0], msg[1]    
    # remove punctuations, only have white-space as delimiter
    words = regex.sub(' ', msg[-1].lower())
    # split the line into words
    words = words.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1        
        if True: # float(prob[word][0])>0 and float(prob[word][1])>0:
            print '%s\t%s\t%s\t%s' % (msgID, prob[word][0], prob[word][1], isSpam)

Overwriting mapper_c.py


###Reducer (classification)

In [192]:
%%writefile reducer_c.py
#!/usr/bin/python
from operator import itemgetter
import sys, operator, math
import numpy as np

current_msg = None
current_prob = [0, 0] # [math.log(.56), math.log(.44)]
msgID = None
n_error = 0
n_msg = 0
n_zero = [0, 0]
prio = [0, 0] # [math.log(.56), math.log(.44)]

print '%s\t%s\t%s' %('TRUTH', 'PREDICTION', 'EMAIL ID')
# input comes from STDIN
for line in sys.stdin:
    print line
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper_c.py
    msgID = line.split('\t')[0]
    
    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_msg == msgID:        
        current_prob = np.sum([[math.log(x) if x>0 else float('-inf') for x in prob], current_prob], 0)
        msgID, p0, p1, isSpam = line.split('\t', 3)
        prob = [float(p0), float(p1)]
        # convert count and spam flag (currently a string) to int
        try:        
            isSpam = int(isSpam)
        except ValueError:
            # count was not a number, so silently
            # ignore/discard this line
            continue
    else:
        if current_msg:
            # count finish for one word, predict and print result
            pred = np.argmax(current_prob)
            n_error += pred != isSpam
            n_msg += 1
            n_zero[isSpam] += float('-inf') in current_prob
            print '%s\t%s\t%s\t%s' %(isSpam, pred, str(current_prob), current_msg)
        else:
            # initialize the 
            
        # initialize new count for new word
        current_prob = np.sum([[math.log(x) if x>0 else float('-inf') for x in prob], prio], 0)
        current_msg = msgID

# do not forget to print the last msg result if needed!
if current_msg == msgID:
    pred = np.argmax(current_prob)
    n_error += pred != isSpam
    n_msg += 1
    n_zero[isSpam] += float('-inf') in current_prob
    print '%s\t%s\t%s\t%s' %(isSpam, pred, str(current_prob), msgID)
    
# calculate the overall error rate
print 'Error rate: %.4f' %(1.0*n_error/n_msg)
print 'Number of zero probability: spam(%d), ham(%d)' %(n_zero[1], n_zero[0])

Overwriting reducer_c.py


###Run the jobs with Hadoop Streaming

In [193]:
# clearn up HDFS
!hdfs dfs -rm -r results
!hdfs dfs -rm -r prob
# run training job
!hadoop jar /usr/local/Cellar/hadoop/2.*/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar -mapper mapper_t.py -reducer reducer_t.py -input /user/lei/enronemail_1h.txt -output prob
# run classification job
!hadoop jar /usr/local/Cellar/hadoop/2.*/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar -mapper mapper_c.py -reducer reducer_c.py -input /user/lei/enronemail_1h.txt -output results

16/01/23 14:41:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 14:41:34 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted results
16/01/23 14:41:35 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 14:41:36 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted prob
16/01/23 14:41:37 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 14:41:38 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/23 14:41:38 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/23 14:41:38 INFO jvm.JvmMetrics

###<span style="color:red">HW2.3 Results: </span>

In [194]:
!hdfs dfs -cat results/part-00000

16/01/23 14:42:03 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
TRUTH	PREDICTION	EMAIL ID
0001.1999-12-10.farmer	6.98519139424e-05	0.000370625297824	0
	
0001.1999-12-10.farmer	6.98519139424e-05	0.0	0
	
0001.1999-12-10.farmer	6.98519139424e-05	5.29464711177e-05	0
	
0001.1999-12-10.farmer	0.00139703827885	0.0	0
	
0001.1999-12-10.farmer	0.000139703827885	0.000158839413353	0
	
0001.1999-12-10.kaminski	0.00698519139424	0.0182665325356	0
	
0	0	[-44.15678591         -inf]	0001.1999-12-10.kaminski
0001.1999-12-10.kaminski	0.000349259569712	0.000211785884471	0
	
0001.1999-12-10.kaminski	6.98519139424e-05	0.0	0
	
0001.1999-12-10.kaminski	0.00160659402068	0.000529464711177	0
	
0001.2000-01-17.beck	6.98519139424e-05	0.0	0
	
0	0	[-28.92642997         -inf]	0001.2000-01-17.beck
0001.2000-01-17.beck	6.98519139424e-05	0.0	0
	
0001.2000-01-17.beck	0.026683431126	0.0	0
	
0001.2000-01-17.beck	0.026683431126	0.0	0
	
0001.2

###*HW2.4* Repeat HW2.3 with the following modification: 
- use Laplace plus-one smoothing,
- compare the misclassifcation error rates for 2.3 versus 2.4 and explain the differences.
   

###Reducer (training)
- minor change: assign 1 to the smooth_factor (instead of 0 in HW2.3)

In [152]:
%%writefile reducer_t.py
#!/usr/bin/python
from operator import itemgetter
import sys, operator
import numpy as np

current_word = None
smooth_factor = 1 # Laplace plus-one smoothing
current_count = [smooth_factor, smooth_factor]
word = None
wordcount = {}

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count, isSpam = line.split('\t', 2)

    # convert count and spam flag (currently a string) to int
    try:
        count = int(count)
        isSpam = int(isSpam)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:        
        current_count[isSpam] += count
    else:
        if current_word:
            # count finish for one word, save it
            wordcount[current_word] = current_count
        # initialize new count for new word
        current_count = [smooth_factor, smooth_factor]
        current_count[isSpam] = count
        current_word = word

# do not forget to save the last word count if needed!
if current_word == word:    
    wordcount[current_word] = current_count
    
# calculate NB parameters, and write the dictionary to a file for the classification job
n_total = np.sum(wordcount.values(), 0)
# print probability
for (key,value) in zip(wordcount.keys(), wordcount.values()/(1.0*n_total)):
    print '%s\t%s\t%s' %(key, value[0], value[1])

Overwriting reducer_t.py


###Run the jobs with Hadoop Streaming

In [154]:
# clearn up HDFS
!hdfs dfs -rm -r results
!hdfs dfs -rm -r prob
# run training job
!hadoop jar /usr/local/Cellar/hadoop/2*/libexec/share/hadoop/tools/lib/hadoop-streaming-2*.jar -mapper mapper_t.py -reducer reducer_t.py -input /user/lei/enronemail_1h.txt -output prob
# run classification job
!hadoop jar /usr/local/Cellar/hadoop/2*/libexec/share/hadoop/tools/lib/hadoop-streaming-2*.jar -mapper mapper_c.py -reducer reducer_c.py -input /user/lei/enronemail_1h.txt -output results

16/01/23 14:05:35 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 14:05:35 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted results
16/01/23 14:05:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 14:05:37 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted prob
16/01/23 14:05:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 14:05:39 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/23 14:05:39 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/23 14:05:39 INFO jvm.JvmMetrics

###<span style="color:red">HW2.4 Results: </span>

In [155]:
!hdfs dfs -cat results/part-00000

16/01/23 14:05:52 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
TRUTH	PREDICTION	EMAIL ID
0	0	[-45.06667897 -46.56748268]	0001.1999-12-10.kaminski
0	0	[-30.13255398 -31.01072955]	0001.2000-01-17.beck
0	0	[-3730.86538538 -4262.28267488]	0001.2000-06-06.lokay
0	0	[-3749.30290917 -4134.32491514]	0001.2001-02-07.kitchen
0	0	[-344.97288532 -398.74287511]	0001.2001-04-02.williams
0	0	[-1423.69744127 -1430.58093544]	0002.1999-12-13.farmer
0	0	[-2993.00697988 -3432.96921054]	0002.2001-02-07.kitchen
1	0	[-453.56808731 -459.84677075]	0002.2001-05-25.SA_and_HP
1	1	[-660.48254994 -613.16498577]	0002.2003-12-18.GP
1	1	[-1399.31068208 -1311.49142952]	0002.2004-08-01.BG
0	1	[-898.0546497  -853.22086743]	0003.1999-12-10.kaminski
0	0	[-422.40471894 -473.12939468]	0003.1999-12-14.farmer
0	0	[-80.34520006 -95.2407242 ]	0003.2000-01-17.beck
0	0	[-1304.21554516 -1524.26333677]	0003.2001-02-08.kitchen
1	0	[-1383.30596029 -15

###<span style="color:red">HW2.4 Discussions: </span>

###*HW2.5.* Repeat HW2.4. 
- This time when modeling and classification ignore tokens with a frequency of less than three (3) in the training set. 
- How does it affect the misclassifcation error of learnt naive multinomial Bayesian Classifier on the training dataset:

###Reducer (training)
- minor change with previous: at the end not emitting word~probability pair with less than 3 count

In [123]:
%%writefile reducer_t.py
#!/usr/bin/python
from operator import itemgetter
import sys, operator
import numpy as np

current_word = None
smooth_factor = 0 # no smoothing
current_count = [smooth_factor, smooth_factor]
word = None
wordcount = {}

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count, isSpam = line.split('\t', 2)

    # convert count and spam flag (currently a string) to int
    try:
        count = int(count)
        isSpam = int(isSpam)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:        
        current_count[isSpam] += count
    else:
        if current_word:
            # count finish for one word, save it
            wordcount[current_word] = current_count
        # initialize new count for new word
        current_count = [smooth_factor, smooth_factor]
        current_count[isSpam] = count
        current_word = word

# do not forget to save the last word count if needed!
if current_word == word:    
    wordcount[current_word] = current_count
    
# calculate NB parameters, and write the dictionary to a file for the classification job
n_total = np.sum(wordcount.values(), 0)
# print probability
for (key,value) in zip(wordcount.keys(), wordcount.values()/(1.0*n_total)):
    # only emit probability when the count (spam and ham together) is no less than 3
    if sum(wordcount[key]) >= 3:
        print '%s\t%s\t%s' %(key, value[0], value[1])

Overwriting reducer_t.py


###Mapper (classification)
- change: at the end only emit word which has probability from training step

In [124]:
%%writefile mapper_c.py
#!/usr/bin/python
import sys, re, string, subprocess
# read the probability from HDFS
prob = {}
cat = subprocess.Popen(["hadoop", "fs", "-cat", "prob/part-00000"], stdout=subprocess.PIPE)
for line in cat.stdout:
    word, p0, p1 = line.split()
    prob[word] = [p0, p1]

# define regex for punctuation removal
regex = re.compile('[%s]' % re.escape(string.punctuation))
# input comes from STDIN (standard input)
for line in sys.stdin:
    # use subject and body
    msg = line.split('\t', 2)
    # skip bad message 
    if len(msg) < 3:
        continue
    msgID, isSpam = msg[0], msg[1]    
    # remove punctuations, only have white-space as delimiter
    words = regex.sub(' ', msg[-1].lower())
    # split the line into words
    words = words.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        if word in prob:
            print '%s\t%s\t%s\t%s' % (msgID, prob[word][0], prob[word][1], isSpam)

Overwriting mapper_c.py


###Run the jobs with Hadoop Streaming

In [125]:
# clearn up HDFS
!hdfs dfs -rm -r results
!hdfs dfs -rm -r prob
# run training job
!hadoop jar /usr/local/Cellar/hadoop/2*/libexec/share/hadoop/tools/lib/hadoop-streaming-2*.jar -mapper mapper_t.py -reducer reducer_t.py -input /user/lei/enronemail_1h.txt -output prob
# run classification job
!hadoop jar /usr/local/Cellar/hadoop/2*/libexec/share/hadoop/tools/lib/hadoop-streaming-2*.jar -mapper mapper_c.py -reducer reducer_c.py -input /user/lei/enronemail_1h.txt -output results

16/01/23 13:25:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 13:25:16 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted results
16/01/23 13:25:17 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 13:25:17 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted prob
16/01/23 13:25:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/23 13:25:19 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/23 13:25:19 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/23 13:25:19 INFO jvm.JvmMetrics

###<span style="color:red">HW2.5 Results: </span>

In [126]:
!hdfs dfs -cat results/part-00000

16/01/23 13:25:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
TRUTH	PREDICTION	EMAIL ID
0	0	[-25.01851976         -inf]	0001.1999-12-10.kaminski
0	0	[-19.3572969  -20.00626353]	0001.2000-01-17.beck
0	0	[-2913.26208693           -inf]	0001.2000-06-06.lokay
0	0	[-3207.88214833           -inf]	0001.2001-02-07.kitchen
0	0	[-300.70764412          -inf]	0001.2001-04-02.williams
0	0	[-1092.27824087           -inf]	0002.1999-12-13.farmer
0	0	[-2388.17253477           -inf]	0002.2001-02-07.kitchen
1	0	[-350.39030061          -inf]	0002.2001-05-25.SA_and_HP
1	1	[         -inf -431.53702815]	0002.2003-12-18.GP
1	1	[         -inf -802.82675824]	0002.2004-08-01.BG
0	1	[         -inf -678.36247214]	0003.1999-12-10.kaminski
0	0	[-355.30263248          -inf]	0003.1999-12-14.farmer
0	0	[-77.75667841         -inf]	0003.2000-01-17.beck
0	0	[-1114.38617194           -inf]	0003.2001-02-08.kitchen
1	0	[-1173.68297403      

###*HW2.6.* Benchmark your code with the Python SciKit-Learn implementation of the multinomial Naive Bayes algorithm
- Run the Multinomial Naive Bayes algorithm (using default settings) from SciKit-Learn over the same training data used in HW2.5 and report the misclassification error (please note some data preparation might be needed to get the Multinomial Naive Bayes algorithm from SkiKit-Learn to run over this dataset)
- Prepare a table to present your results, where rows correspond to approach used (SkiKit-Learn versus your Hadoop implementation) and the column presents the training misclassification error
- Explain/justify any differences in terms of training error rates over the dataset in HW2.5 between your Multinomial Naive Bayes implementation (in Map Reduce) versus the Multinomial Naive Bayes implementation in SciKit-Learn 


In [19]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import *

import csv
import numpy as np

# read email message, and organize training data
with open('enronemail_1h.txt', 'r') as f:
    reader = csv.reader(f, delimiter="\t")
    emails = list(reader)
train_label = [msg[1] for msg in emails]
train_data = [msg[2]+msg[3] if len(msg)==4 else msg[2] for msg in emails]
msg_id = [msg[0].lower() for msg in emails]

# feature vectorization
uniVectorizer = CountVectorizer()
dtmTrain = uniVectorizer.fit_transform(train_data) 

# multinomial Naive Bayes Classifier from sklearn
mnb = MultinomialNB()
mnb.fit(dtmTrain, train_label)
pred_mnb = mnb.predict(dtmTrain)
training_error_mnb = 1.0*sum(pred_mnb != train_label) / len(train_label)

# Bernoulli Naive Bayes Classifier from sklearn
bnb = BernoulliNB()
bnb.fit(dtmTrain, train_label)
pred_bnb = bnb.predict(dtmTrain)
training_error_bnb = 1.0*sum(pred_bnb != train_label) / len(train_label)

# multinomial Naive Bayes Classifier from HW1.5
!./pNaiveBayes.sh 4 "dummy"

# load results from HW1.5 and generate comparison matrix
print 'TRUTH \t MNB_HW1.5 \t MNB_SK \t BNB_SK \t ID'
with open ('enronemail_1h.txt.output', "r") as myfile:  
    for line in myfile.readlines():
        if line.startswith('ham') or line.startswith('spam'):
            result = line.split()            
            idx = msg_id.index(result[2])
            result.insert(2, 'spam' if pred_mnb[idx]=='1' else 'ham')
            result.insert(3, 'spam' if pred_bnb[idx]=='1' else 'ham')
            print str.join('\t', result)
            
        if line.startswith('Our multinomial NB'):
            print '\n' + line.strip('\n')     

print 'SK- multinomial NB training error: %f' %training_error_mnb
print 'SK- Bernoulli   NB training error: %f' %training_error_bnb

TRUTH 	 MNB_HW1.5 	 MNB_SK 	 BNB_SK 	 ID
ham	ham	ham	ham	0001.1999-12-10.farmer
ham	ham	ham	ham	0001.1999-12-10.kaminski
ham	ham	ham	ham	0001.2000-01-17.beck
ham	ham	ham	ham	0001.2000-06-06.lokay
ham	ham	ham	ham	0001.2001-02-07.kitchen
ham	ham	ham	ham	0001.2001-04-02.williams
ham	ham	ham	ham	0002.1999-12-13.farmer
ham	ham	ham	ham	0002.2001-02-07.kitchen
spam	spam	spam	ham	0002.2001-05-25.sa_and_hp
spam	spam	spam	spam	0002.2003-12-18.gp
spam	spam	spam	ham	0002.2004-08-01.bg
ham	ham	ham	ham	0003.1999-12-10.kaminski
ham	ham	ham	ham	0003.1999-12-14.farmer
ham	ham	ham	ham	0003.2000-01-17.beck
ham	ham	ham	ham	0003.2001-02-08.kitchen
spam	spam	spam	ham	0003.2003-12-18.gp
spam	spam	spam	ham	0003.2004-08-01.bg
ham	ham	ham	ham	0004.1999-12-10.kaminski
ham	ham	ham	ham	0004.1999-12-14.farmer
ham	ham	ham	ham	0004.2001-04-02.williams
spam	spam	spam	spam	0004.2001-06-12.sa_and_hp
spam	spam	spam	ham	0004.2004-08-01.bg
ham	ham	ham	ham	0005.1999-12-12.kaminski
ham	ham	ham	ham	0005.1999-12-14.farmer
ham	

###*HW 2.6.1. OPTIONAL* (note this exercise is a stretch HW and optional)
- Run the Bernoulli Naive Bayes algorithm from SciKit-Learn (using default settings) over the same training data used in HW2.6 and report the misclassification error 
- Discuss the performance differences in terms of misclassification error rates over the dataset in HW2.5 between the  Multinomial Naive Bayes implementation in SciKit-Learn with the  Bernoulli Naive Bayes implementation in SciKit-Learn. Why such big differences. Explain. 
- Which approach to Naive Bayes would you recommend for SPAM detection? Justify your selection.


###*HW2.7. OPTIONAL* (note this exercise is a stretch HW and optional)

The Enron SPAM data in the following folder enron1-Training-Data-RAW is in raw text form (with subfolders for SPAM and HAM that contain raw email messages in the following form:

- Line 1 contains the subject
- The remaining lines contain the body of the email message.

In Python write a script to produce a TSV file called train-Enron-1.txt that has a similar format as the enronemail_1h.txt that you have been using so far. Please pay attend to funky characters and tabs. Check your resulting formated email data in Excel and in Python (e.g., count up the number of fields in each row; the number of SPAM mails and the number of HAM emails). Does each row correspond to an email record with four values? Note: use "NA" to denote empty field values.

###*HW2.8.*
Using Hadoop Map-Reduce write job(s) to perform the following:
- Train a multinomial Naive Bayes Classifier with Laplace plus one smoothing using the data extracted in HW2.7 (i.e., train-Enron-1.txt). Use all white-space delimitted tokens as independent input variables (assume spaces, fullstops, commas as delimiters). Drop tokens with a frequency of less than three (3).
- Test the learnt classifier using enronemail_1h.txt and report the misclassification error rate. Remember to use all white-space delimitted tokens as independent input variables (assume spaces, fullstops, commas as delimiters). How do we treat tokens in the test set that do not appear in the training set?

###*HW2.8.1.*
- Run  both the Multinomial Naive Bayes and the Bernoulli Naive Bayes algorithms from SciKit-Learn (using default settings) over the same training data used in HW2.8 and report the misclassification error on both the training set and the testing set
- Prepare a table to present your results, where rows correspond to approach used (SciKit-Learn Multinomial NB; SciKit-Learn Bernouili NB; Your Hadoop implementation)  and the columns presents the training misclassification error, and the misclassification error on the test data set
- Discuss the performance differences in terms of misclassification error rates over the test and training datasets by the different implementations. Which approch (Bernouili versus Multinomial) would you recommend for SPAM detection? Justify your selection.



###stop yarn and hdfs

In [195]:
!/usr/local/Cellar/hadoop/2*/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2*/sbin/stop-dfs.sh

stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
no proxyserver to stop
16/01/23 15:01:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
16/01/23 15:02:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
